In [21]:
import numpy as np

recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

prices_recommended = [100, 90, 10, 450, 50, 37, 99, 120, 34, 100]
prices_bought = [110, 190, 100, 450]

In [22]:
#был ли хотя бы 1 релевантный товар среди топ-k рекомендованных
def hit_rate_at_k(recommended_list, bought_list, k):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate

#проверим
print(hit_rate_at_k(recommended_list, bought_list, 5))

1


In [23]:
#(revenue of recommended items @k that are relevant) / (revenue of recommended items @k)
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k):
        
    # your_code
    # Лучше считать через скалярное произведение, а не цикл
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = np.array(recommended_list[:k])
    prices_recommended = np.array(prices_recommended[:k])
    
    flags = np.isin(recommended_list, bought_list) #вернет размерность recommended_list, т.е. k
    
    precision = np.dot(flags, prices_recommended) / prices_recommended.sum()
    
    
    return precision

#проверим
print(money_precision_at_k(recommended_list, bought_list, prices_recommended, 5))

0.7857142857142857


In [24]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [25]:
print(recall(recommended_list, bought_list))

0.5


In [26]:
#Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

def recall_at_k(recommended_list, bought_list, k):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
  
    return recall

#проверим
print(recall_at_k(recommended_list, bought_list,3))
print(recall_at_k(recommended_list, bought_list,5))

0.25
0.5


In [27]:
#Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items) 

def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k):
    bought_list = np.array(bought_list)
    prices_bought = np.array(prices_bought)
    recommended_list = np.array(recommended_list[:k])
    prices_recommended = np.array(prices_recommended[:k])
    
    flags = np.isin(recommended_list, bought_list)
    
    r1 = np.dot(flags,prices_recommended)
    r2 = prices_bought.sum()
    recall = r1 / r2
  
    return recall

#проверим

print(money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought,1))
print(money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought,3))
print(money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought,4))
print(money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought,5))

0.11764705882352941
0.11764705882352941
0.6470588235294118
0.6470588235294118


In [28]:
## MRR
def reciprocal_rank(recommended_list, bought_list):
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    count = 0
    for i in range(1, len(flags)+1):
        if flags[i-1]:
            sum_ += 1/i
            print(f'iteration:{i}',f'rank (1/i):{1/i}')
            count += 1
    result = sum_ / count
    
    return result

In [29]:
print(reciprocal_rank(recommended_list, bought_list))

iteration:1 rank (1/i):1.0
iteration:4 rank (1/i):0.25
0.625


In [42]:
## MAP@k (Mean Average Precision@k) - среднее AP@k по всем юзерам.
def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)


In [43]:
def ap_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    # np.nonzero() - get indexes of nonzero elements of array, returns a tuple with index array as 0th member
    rec_and_relev_idx = np.nonzero(np.isin(recommended_list, bought_list))[0]
    rec_and_relev_count = len(rec_and_relev_idx)
    if rec_and_relev_count == 0:
        return 0
    
    precision_sum = sum([precision_at_k(recommended_list, bought_list, k=idx+1) for idx in rec_and_relev_idx])
    return precision_sum / rec_and_relev_count

In [44]:
ap_k(recommended_list, bought_list, k=4)

0.75

In [49]:
def map_k(recommended_list, bought_list, k=5):
    ap_k_sum = sum([ap_k(user_rec_list, user_bought_list, k=k)
                    for user_rec_list, user_bought_list in zip(recommended_list, bought_list)])
    return ap_k_sum / len(recommended_list)

In [50]:
# 3 пользователя
rec_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],   # user 1
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],        # user 2
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]]   # user 3

bought_list_3_users = [[521, 32, 143],            # user 1
                       [143, 156, 991, 43, 11],   # user 2
                       [1, 2]]                    # user 3

In [51]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1):
        
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

In [52]:
ap_k(recommended_list, bought_list, k=5)

0.16666666666666666

In [53]:
def map_k(recommended_list, bought_list, k=5):
    ap_k_sum = sum([ap_k(user_rec_list, user_bought_list, k=k)
                    for user_rec_list, user_bought_list in zip(recommended_list, bought_list)])
    return ap_k_sum / len(recommended_list)

In [54]:
rec_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],   # user 1
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],        # user 2
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]]   # user 3

bought_list_3_users = [[521, 32, 143],            # user 1
                       [143, 156, 991, 43, 11],   # user 2
                       [1, 2]]                    # user 3

In [60]:
map_k(rec_list_3_users, bought_list_3_users, k=4)

0.0